# This code base is developed to validate a concept that DDLs can be created from Excel file

In [ ]:
pip install pandas

In [9]:
import pandas as pd

df = pd.read_csv('data/snowflake.csv')
# df = pd.read_csv('data/redshift.csv')
# df = pd.read_csv('data/qvds.csv')

print(df.to_string()) 

  database   schema      table  column_name  datatype
0  dev_raw   cerner   my_table         col1   varchar
1  dev_raw   cerner   my_table         col2       int
2  dev_raw   cerner   my_table         col3     float
3  dev_raw   cerner   my_table         col4   varchar


In [31]:
import csv

def generate_ddl(csv_file):
    ddl_statements = {}
    with open(csv_file, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            database_name = row['database'].strip()
            table_name = row['schema'].strip()
            schema_name = row['table'].strip()
            column_name = row['column_name'].strip()
            data_type = row['data_type'].strip()
            
            print(f"generating table/view ddl for {schema_name}.{table_name}")
            ddl = f"CREATE TABLE {schema_name}.{table_name} (\n"
            ddl += f"\t{column_name} {data_type},\n"

            if (schema_name, table_name) not in ddl_statements:
                ddl_statements[(schema_name, table_name)] = ddl
            else:
                ddl_statements[(schema_name, table_name)] += f"\t{column_name} {data_type},\n"
    
    for key, value in ddl_statements.items():
        ddl_statements[key] = value.rstrip(',\n') + "\n);"
    
    return ddl_statements

def write_ddl_files(ddl_statements, output_folder):
    for key, value in ddl_statements.items():
        schema_name, table_name = key
        file_name = f"{output_folder}/crt_{table_name}.sql"
        print(f"writing ddl for {schema_name}.{table_name} to {file_name} with content:\n{value}")
        with open(file_name, 'w') as file:
            file.write(value)
        print(f"Done writing ddl for {schema_name}.{table_name} to {file_name}")
        
csv_file = 'data/snowflake.csv'
output_folder = 'output/snowflake/'

ddl_statements = generate_ddl(csv_file)
write_ddl_files(ddl_statements, output_folder)

generating table/view ddl for my_table.cerner
CREATE TABLE my_table.cerner (
	col1 varchar,

generating table/view ddl for my_table.cerner
CREATE TABLE my_table.cerner (
	col2 int,

generating table/view ddl for my_table.cerner
CREATE TABLE my_table.cerner (
	col3 float,

generating table/view ddl for my_table.cerner
CREATE TABLE my_table.cerner (
	col4 varchar,

writing ddl for my_table.cerner to output/snowflake//crt_cerner.sql with content:
CREATE TABLE my_table.cerner (
	col1 varchar,
	col2 int,
	col3 float,
	col4 varchar
);
Done writing ddl for my_table.cerner to output/snowflake//crt_cerner.sql
